In [1]:
import glob
import numpy as np
import pandas as pd 
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json 

import certifi
import ssl
import geopy.geocoders
from geopy.geocoders import Nominatim 
ctx = ssl.create_default_context(cafile=certifi.where())
geopy.geocoders.options.default_ssl_context = ctx

import requests 
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
import folium # map rendering library


In [2]:

path = r'Venues' # use your path
all_files = glob.glob(path + "/*.csv")

li = []

for filename in all_files:
    df = pd.read_csv(filename, index_col=0, header=0)
    li.append(df)

df_venues = pd.concat(li, axis=0, ignore_index=False)
df_venues.shape

(44306, 7)

In [3]:
# df_venues.to_csv('venues.csv')

In [4]:
# df_venues = pd.read_csv('venues.csv',index_col=0, header=0)
df_venues.head()


,Address,Suburb Latitude,Suburb Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Ashok Vihar, New Delhi India",28.699453,77.184826,"Green Chick Chop, Deep Market",28.701142,77.186672,Butcher
1,"Ashok Vihar, New Delhi India",28.699453,77.184826,Bakers Stop,28.700495,77.188716,Bakery
2,"Ashok Vihar, New Delhi India",28.699453,77.184826,Invitation Banquet,28.696018,77.185953,Diner
3,"Ashok Vihar, New Delhi India",28.699453,77.184826,Gola Northend,28.701242,77.189288,Indian Restaurant
4,"Bawana, New Delhi India",28.799660,77.032885,Rajiv Gandhi Stadium,28.801811,77.028751,Playground


In [5]:
# number of venues were returned for each neighborhood
# df_venues.groupby('Address').count()

In [6]:
# number of unique categories can be curated from all the returned venues
print('There are {} uniques categories.'.format(len(df_venues['Venue Category'].unique())))

There are 577 uniques categories.


In [60]:
# Analyze Each Address

# one hot encoding
venues_onehot = pd.get_dummies(df_venues[['Venue Category']], prefix="", prefix_sep="")

# add Address column back to dataframe
venues_onehot['Address'] = df_venues['Address']

# move Address column to the first column
fixed_columns = [venues_onehot.columns[-1]] + list(venues_onehot.columns[:-1])
venues_onehot = venues_onehot[fixed_columns]
print(venues_onehot.shape)
venues_onehot.head()

(44306, 578)


,Address,ATM,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,Airport,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Amphitheater,Animal Shelter,Antique Shop,Aquarium,Arcade,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Astrologer,Athletics & Sports,Auditorium,Australian Restaurant,Austrian Restaurant,Auto Dealership,Auto Garage,Auto Workshop,Automotive Shop,BBQ Joint,Baby Store,Badminton Court,Bagel Shop,Bakery,Bangladeshi Restaurant,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Bath House,Bay,Beach,Beach Bar,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Beijing Restaurant,Bengali Restaurant,Big Box Store,Bike Rental / Bike Share,Bike Shop,Bike Trail,Bistro,Board Shop,Boat or Ferry,Bookstore,Botanical Garden,Boutique,Bowling Alley,Bowling Green,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bridge,Bubble Tea Shop,Buddhist Temple,Buffet,Building,Bulgarian Restaurant,Burger Joint,Burmese Restaurant,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Cambodian Restaurant,Camera Store,Campground,Canal,Canal Lock,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Carpet Store,Casino,Castle,Caucasian Restaurant,Cemetery,Cha Chaan Teng,Chaat Place,Check Cashing Service,Cheese Shop,Child Care Service,Chinese Restaurant,Chocolate Shop,Church,Churrascaria,Circus,City Hall,Climbing Gym,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College Academic Building,College Arts Building,College Basketball Court,College Bookstore,College Cafeteria,College Football Field,College Gym,College Rec Center,College Residence Hall,College Stadium,College Theater,Colombian Restaurant,Comedy Club,Comfort Food Restaurant,Comic Shop,Community Center,Concert Hall,Confucian Temple,Construction & Landscaping,Convenience Store,Cooking School,Cosmetics Shop,Costume Shop,Coworking Space,Creperie,Cricket Ground,Cruise,Cuban Restaurant,Cultural Center,Cupcake Shop,Cycle Studio,Czech Restaurant,Dance Studio,Daycare,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dhaba,Dim Sum Restaurant,Diner,Disc Golf,Discount Store,Distillery,Dive Bar,Doctor's Office,Dog Run,Donburi Restaurant,Doner Restaurant,Dongbei Restaurant,Donut Shop,Dosa Place,Drugstore,Dry Cleaner,Dumpling Restaurant,Duty-free Shop,Eastern European Restaurant,Egyptian Restaurant,Electronics Store,Emergency Room,Empanada Restaurant,English Restaurant,Entertainment Service,Ethiopian Restaurant,Event Service,Event Space,Exhibit,Eye Doctor,Fabric Shop,Factory,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Film Studio,Financial or Legal Service,Fish & Chips Shop,Fish Market,Fishing Spot,Fishing Store,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Service,Food Stand,Food Truck,Football Stadium,Forest,Fountain,Fraternity House,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Gluten-free Restaurant,Go Kart Track,Golf Course,Golf Driving Range,Gourmet Shop,Government Building,Greek Restaurant,Grocery Store,Guizhou Restaurant,Gun Range,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Hainan Restaurant,Halal Restaurant,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Heliport,Herbs & Spices Store,High School,Himalayan Restaurant,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hong Kong Restaurant,Hookah Bar,Hostel,Hot Dog Joint,Hot Spring,Hotel,Hotel Bar,Hotel Pool,Hotpot Restaurant,Hubei Restaurant,Hunan Restaurant,IT Services,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indian Sweet Shop,Indie Movie Theater,Indie Theater,Indonesian Restaurant,Indoor Play Area,Insurance

In [61]:
# group rows by neighborhood and by taking the mean of the frequency of occurrence of each category
venues_grouped = venues_onehot.groupby('Address').mean().reset_index()
print(venues_grouped.shape)
venues_grouped.head()


(1117, 578)


,Address,ATM,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,Airport,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Amphitheater,Animal Shelter,Antique Shop,Aquarium,Arcade,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Astrologer,Athletics & Sports,Auditorium,Australian Restaurant,Austrian Restaurant,Auto Dealership,Auto Garage,Auto Workshop,Automotive Shop,BBQ Joint,Baby Store,Badminton Court,Bagel Shop,Bakery,Bangladeshi Restaurant,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Bath House,Bay,Beach,Beach Bar,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Beijing Restaurant,Bengali Restaurant,Big Box Store,Bike Rental / Bike Share,Bike Shop,Bike Trail,Bistro,Board Shop,Boat or Ferry,Bookstore,Botanical Garden,Boutique,Bowling Alley,Bowling Green,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bridge,Bubble Tea Shop,Buddhist Temple,Buffet,Building,Bulgarian Restaurant,Burger Joint,Burmese Restaurant,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Cambodian Restaurant,Camera Store,Campground,Canal,Canal Lock,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Carpet Store,Casino,Castle,Caucasian Restaurant,Cemetery,Cha Chaan Teng,Chaat Place,Check Cashing Service,Cheese Shop,Child Care Service,Chinese Restaurant,Chocolate Shop,Church,Churrascaria,Circus,City Hall,Climbing Gym,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College Academic Building,College Arts Building,College Basketball Court,College Bookstore,College Cafeteria,College Football Field,College Gym,College Rec Center,College Residence Hall,College Stadium,College Theater,Colombian Restaurant,Comedy Club,Comfort Food Restaurant,Comic Shop,Community Center,Concert Hall,Confucian Temple,Construction & Landscaping,Convenience Store,Cooking School,Cosmetics Shop,Costume Shop,Coworking Space,Creperie,Cricket Ground,Cruise,Cuban Restaurant,Cultural Center,Cupcake Shop,Cycle Studio,Czech Restaurant,Dance Studio,Daycare,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dhaba,Dim Sum Restaurant,Diner,Disc Golf,Discount Store,Distillery,Dive Bar,Doctor's Office,Dog Run,Donburi Restaurant,Doner Restaurant,Dongbei Restaurant,Donut Shop,Dosa Place,Drugstore,Dry Cleaner,Dumpling Restaurant,Duty-free Shop,Eastern European Restaurant,Egyptian Restaurant,Electronics Store,Emergency Room,Empanada Restaurant,English Restaurant,Entertainment Service,Ethiopian Restaurant,Event Service,Event Space,Exhibit,Eye Doctor,Fabric Shop,Factory,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Film Studio,Financial or Legal Service,Fish & Chips Shop,Fish Market,Fishing Spot,Fishing Store,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Service,Food Stand,Food Truck,Football Stadium,Forest,Fountain,Fraternity House,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Gluten-free Restaurant,Go Kart Track,Golf Course,Golf Driving Range,Gourmet Shop,Government Building,Greek Restaurant,Grocery Store,Guizhou Restaurant,Gun Range,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Hainan Restaurant,Halal Restaurant,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Heliport,Herbs & Spices Store,High School,Himalayan Restaurant,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hong Kong Restaurant,Hookah Bar,Hostel,Hot Dog Joint,Hot Spring,Hotel,Hotel Bar,Hotel Pool,Hotpot Restaurant,Hubei Restaurant,Hunan Restaurant,IT Services,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indian Sweet Shop,Indie Movie Theater,Indie Theater,Indonesian Restaurant,Indoor Play Area,Insurance

In [62]:
# venues_grouped.to_csv('venues_grouped.csv')

In [63]:
# sort the venues in descending order.

def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

# create a new dataframe and display the top 10 venues for each Suburb
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Address']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
suburbs_venues_sorted = pd.DataFrame(columns=columns)
suburbs_venues_sorted['Address'] = venues_grouped['Address']

for ind in np.arange(venues_grouped.shape[0]):
    suburbs_venues_sorted.iloc[ind, 1:] = return_most_common_venues(venues_grouped.iloc[ind, :], num_top_venues)

suburbs_venues_sorted.head()

,Address,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Barking and Dagenham, London UK",Grocery Store,Supermarket,Park,Platform,Bus Stop,Gas Station,Pub,Furniture / Home Store,Café,Shopping Mall
1,"Barnet, London UK",Coffee Shop,Supermarket,Café,Turkish Restaurant,Pub,Italian Restaurant,Park,Pizza Place,Pharmacy,Grocery Store
2,"Bexley, London UK",Pub,Grocery Store,Supermarket,Fast Food Restaurant,Coffee Shop,Italian Restaurant,Clothing Store,Hotel,American Restaurant,Indian Restaurant
3,"Brent, London UK",Coffee Shop,Sandwich Place,Indian Restaurant,Grocery Store,Clothing Store,Hotel,Pizza Place,Gym / Fitness Center,Portuguese Restaurant,Pharmacy
4,"Bromley, London UK",Pub,Supermarket,Park,Fruit & Vegetable Store,Lake,Turkish Restaurant,Molecular Gastronomy Restaurant,Coffee Shop,Fast Food Restaurant,Platform


In [64]:
# suburbs_venues_sorted.to_csv('venues_sorted.csv')

In [65]:
# Cluster Suburbs

# set number of clusters
kclusters = 10

venues_grouped_clustering = venues_grouped.drop('Address', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(venues_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_

array([2, 2, 2, ..., 2, 5, 2], dtype=int32)

In [66]:
# dataframe that includes the cluster as well as the top 10 venues for each Suburb

# add clustering labels
suburbs_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)
# merge venues_grouped with venues data to add latitude/longitude for each Suburb
venues_merged = suburbs_venues_sorted.join(df_venues.set_index('Address'), on='Address')
venues_merged.shape # check the last columns!

(44306, 18)

In [67]:
venues_merged[['Suburb','Region']] = venues_merged.Address.str.split(", ", expand=True)
cols = venues_merged.columns.tolist()
cols = cols[:2] + cols[-2:] + cols[2:-2]
venues_merged = venues_merged[cols]
venues_merged.head()

,Cluster Labels,Address,Suburb,Region,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Suburb Latitude,Suburb Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,2,"Barking and Dagenham, London UK",Barking and Dagenham,London UK,Grocery Store,Supermarket,Park,Platform,Bus Stop,Gas Station,Pub,Furniture / Home Store,Café,Shopping Mall,51.554117,0.150504,Central Park,51.559560,0.161981,Park
0,2,"Barking and Dagenham, London UK",Barking and Dagenham,London UK,Grocery Store,Supermarket,Park,Platform,Bus Stop,Gas Station,Pub,Furniture / Home Store,Café,Shopping Mall,51.554117,0.150504,Lara Grill,51.562445,0.147178,Turkish Restaurant
0,2,"Barking and Dagenham, London UK",Barking and Dagenham,London UK,Grocery Store,Supermarket,Park,Platform,Bus Stop,Gas Station,Pub,Furniture / Home Store,Café,Shopping Mall,51.554117,0.150504,wilko,51.541002,0.148898,Furniture / Home Store
0,2,"Barking and Dagenham, London UK",Barking and Dagenham,London UK,Grocery Store,Supermarket,Park,Platform,Bus Stop,Gas Station,Pub,Furniture / Home Store,Café,Shopping Mall,51.554117,0.150504,Hoo Hing,51.567561,0.135999,Grocery Store
0,2,"Barking and Dagenham, London UK",Barking and Dagenham,London UK,Grocery Store,Supermarket,Park,Platform,Bus Stop,Gas Station,Pub,Furniture / Home Store,Café,Shopping Mall,51.554117,0.150504,The Eva Hart (Wetherspoon),51.570460,0.130342,Pub


In [68]:
regions = venues_merged['Region'].unique()
for region in regions:
    index = venues_merged[venues_merged['Region']==region]
    print(region)
    print(index['Cluster Labels'].unique())

London UK
[2 1]
Brisbane
[2 5 6 3 8 1 0 9 4]
Tokyo Japan
[2 5]
Christchurch New Zealand
[5 9 2 6 1 3]
Mumbai City India
[2 5 9]
Sydney
[5 2]
New York City US
[2 9 6 5 1 4 3]
New Delhi India
[1 5 2 9 7 0]
Beijing China
[5 1 2]
Los Angeles US
[2 9 5 3 0 4 1]
Auckland City New Zealand
[5 1 2 9]
Hanoi Vietnam
[2 5 1]
Kathmandu Nepal
[1]
Kuala Lumpur Malaysia
[2 5]
Shanghai China
[1 2 9]
Melbourne
[5 2]


In [69]:
venues_merged.to_csv('venues_merged.csv')

In [70]:
df = venues_merged.drop(columns=['Venue','Venue Latitude','Venue Longitude','Venue Category'])
df.head()

,Cluster Labels,Address,Suburb,Region,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Suburb Latitude,Suburb Longitude
0,2,"Barking and Dagenham, London UK",Barking and Dagenham,London UK,Grocery Store,Supermarket,Park,Platform,Bus Stop,Gas Station,Pub,Furniture / Home Store,Café,Shopping Mall,51.554117,0.150504
0,2,"Barking and Dagenham, London UK",Barking and Dagenham,London UK,Grocery Store,Supermarket,Park,Platform,Bus Stop,Gas Station,Pub,Furniture / Home Store,Café,Shopping Mall,51.554117,0.150504
0,2,"Barking and Dagenham, London UK",Barking and Dagenham,London UK,Grocery Store,Supermarket,Park,Platform,Bus Stop,Gas Station,Pub,Furniture / Home Store,Café,Shopping Mall,51.554117,0.150504
0,2,"Barking and Dagenham, London UK",Barking and Dagenham,London UK,Grocery Store,Supermarket,Park,Platform,Bus Stop,Gas Station,Pub,Furniture / Home Store,Café,Shopping Mall,51.554117,0.150504
0,2,"Barking and Dagenham, London UK",Barking and Dagenham,London UK,Grocery Store,Supermarket,Park,Platform,Bus Stop,Gas Station,Pub,Furniture / Home Store,Café,Shopping Mall,51.554117,0.150504


In [71]:
def clusterMap(address, vLatitude_col, vLongitude_col, address_col, clusterLabel_col, zoom_start):

    geolocator = Nominatim(user_agent="explorer")
    location = geolocator.geocode(address)
    latitude = location.latitude
    longitude = location.longitude
    print('The geograpical coordinate of {} are {}, {}.'.format(address, latitude, longitude))

    # create map
    map_clusters = folium.Map(location=[latitude, longitude], zoom_start=zoom_start)

    # set color scheme for the clusters
    x = np.arange(kclusters)
    ys = [i + x + (i*x)**2 for i in range(kclusters)]
    colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
    rainbow = [colors.rgb2hex(i) for i in colors_array]

    # add markers to the map
    markers_colors = []
    for lat, lon, poi, cluster in zip(vLatitude_col, vLongitude_col, address_col, clusterLabel_col):
        label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
        folium.CircleMarker(
            [lat, lon],
            radius=5,
            popup=label,
            color=rainbow[cluster-1],
            fill=True,
            fill_color=rainbow[cluster-1],
            fill_opacity=0.7).add_to(map_clusters)
        
    return(map_clusters)

In [72]:
index = venues_merged[venues_merged['Region']=='Sydney']
clusterMap('City of Sydney, Australia', index['Venue Latitude'], index['Venue Longitude'], index['Address'], index['Cluster Labels'], 12)

The geograpical coordinate of City of Sydney, Australia are -33.8853222, 151.2065221.
